<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

## Sentinel-1 TOPS stack processor
The detailed algorithm for stack processing of TOPS data can be find here:

+ Fattahi, H., P. Agram, and M. Simons (2016), A Network-Based Enhanced Spectral Diversity Approach for TOPS Time-Series Analysis, IEEE Transactions on Geoscience and Remote Sensing, 55(2), 777-786, doi:[10.1109/TGRS.2016.2614925](https://ieeexplore.ieee.org/abstract/document/7637021).

-----------------------------------

The scripts provides support for Sentinel-1 TOPS stack processing. Currently supported workflows include a coregistered stack of SLC, interferograms, offsets, and coherence. 


##To use the sentinel stack processor, make sure to add the path of your `contrib/stack/topsStack` folder to your `$PATH` environment varibale. 

#### Be sure [default] credentials in ~/.aws/credentials are valid


### Begin by loading required libraries and dependencies - already installed in the on-demand system

In [ ]:
from math import floor, ceil
import json

import osaka
import osaka.main
import zipfile
import numpy as np
import gdal
from builtins import str
import os, sys, re, json, logging, traceback, requests, argparse
from datetime import datetime
from pprint import pformat
from matplotlib import pyplot as plt
from requests.packages.urllib3.exceptions import (InsecureRequestWarning,
                                                  InsecurePlatformWarning)
try: from html.parser import HTMLParser
except: from html.parser import HTMLParser
        
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
requests.packages.urllib3.disable_warnings(InsecurePlatformWarning)

# this block makes sure the directory set-up/change is only done once and relative to the notebook's directory\n",
try:
    start_dir
except NameError:
    start_dir = os.getcwd()
    output_dir = os.path.join(start_dir, 'notebook_output/topsStackIfgExample')
    work_dir = os.path.join(output_dir, 'LVC_stack_small')
    os.makedirs(output_dir, exist_ok=True)

os.chdir(output_dir)  

In [ ]:
SLC_RE = re.compile(r'(?P<mission>S1\w)_IW_SLC__.*?' +
                    r'_(?P<start_year>\d{4})(?P<start_month>\d{2})(?P<start_day>\d{2})' +
                    r'T(?P<start_hour>\d{2})(?P<start_min>\d{2})(?P<start_sec>\d{2})' +
                    r'_(?P<end_year>\d{4})(?P<end_month>\d{2})(?P<end_day>\d{2})' +
                    r'T(?P<end_hour>\d{2})(?P<end_min>\d{2})(?P<end_sec>\d{2})_.*$')
ISCE_HOME="/opt/isce2/isce"

ORBITMAP = [('precise','aux_poeorb', 100),
            ('restituted','aux_resorb', 100)]

OPER_RE = re.compile(r'S1\w_OPER_AUX_(?P<type>\w+)_OPOD_(?P<yr>\d{4})(?P<mo>\d{2})(?P<dy>\d{2})')


### Define your dataset and a few processing parameters

In [ ]:
global runtime_dict
runtime_dict = {}
ISCE_HOME="/opt/isce2/isce"
ctx = {}

ctx["min_lat"] = ""
ctx["max_lat"] = ""
ctx["min_lon"] = ""
ctx["max_lon"] = ""

ctx["master_date"]="20180103"
ctx["localize_slcs"]= ["S1A_IW_SLC__1SDV_20180103T135924_20180103T135951_019991_0220D6_3C80", 
                       "S1A_IW_SLC__1SDV_20180421T135924_20180421T135951_021566_0252AE_60EA",
                       "S1A_IW_SLC__1SDV_20180714T135929_20180714T135956_022791_027880_9FCA",
                       "S1A_IW_SLC__1SDV_20181018T135933_20181018T140000_024191_02A573_AEBB",
                       "S1A_IW_SLC__1SDV_20190110T135931_20190110T135958_025416_02D0B7_C555",
                       "S1A_IW_SLC__1SDV_20190416T135930_20190416T135957_026816_03038D_4177",
                       "S1A_IW_SLC__1SDV_20190721T135936_20190721T140003_028216_032FFD_1F8B",
                       "S1A_IW_SLC__1SDV_20191013T135939_20191013T140006_029441_035951_9DBD",
                       "S1A_IW_SLC__1SDV_20191212T135938_20191212T140005_030316_0377AE_ED8B"
                     ]
ctx["work_dir"] = work_dir
wd = ctx["work_dir"]


### Load some helpful modules

In [ ]:
import requests
import re
import os
from datetime import datetime, timedelta
import urllib.parse

def session_get(session, url):
    return session.get(url, verify=False)

datefmt = "%Y%m%dT%H%M%S"
queryfmt = "%Y-%m-%d"
server = 'https://scihub.copernicus.eu/gnss/'
auth_netloc_fmt = "{}:{}@{}"
url_tpt = 'search?q=( beginPosition:[{0}T00:00:00.000Z TO {1}T23:59:59.999Z] AND endPosition:[{0}T00:00:00.000Z TO {1}T23:59:59.999Z] ) AND ( (platformname:Sentinel-1 AND filename:{2}_* AND producttype:{3}))&start=0&rows=100'
credentials = ('gnssguest','gnssguest')

class MyHTMLParser(HTMLParser):

    def __init__(self,url):
        HTMLParser.__init__(self)
        self.fileList = []
        self._url = url

    def handle_starttag(self, tag, attrs):
        for name, val in attrs:
            if name == 'href':
                if val.startswith("https://scihub.copernicus.eu/gnss/odata") and val.endswith(")/"):
                    pass
                else:
                    downloadLink = val.strip()
                    downloadLink = downloadLink.split("/Products('Quicklook')")
                    downloadLink = downloadLink[0] + downloadLink[-1]
                    self._url = downloadLink

    def handle_data(self, data):
        if data.startswith("S1") and data.endswith(".EOF"):
            self.fileList.append((self._url, data.strip()))


def fileToRange(fname):
    '''
    Derive datetime range from orbit file name.
    '''

    fields = os.path.basename(fname).split('_')
    start = datetime.strptime(fields[-2][1:16], datefmt)
    stop = datetime.strptime(fields[-1][:15], datefmt)
    mission = fields[0]

    return (start, stop, mission)


def get_download_orbit_dict(slc_start_dt, slc_end_dt, sat_name):

    found = False
    delta = timedelta(days=1)
    timebef = (slc_end_dt - delta).strftime(queryfmt)
    timeaft = (slc_end_dt + delta).strftime(queryfmt)
    match = None
    matchFileName = None
    
    session = requests.Session()

    for fidelity in ('AUX_POEORB', 'AUX_RESORB'):
        url = server + url_tpt.format(timebef, timeaft, sat_name, fidelity)
        # print(f"url: {url}")

        try:
            r = session.get(url, verify=True, auth=credentials)
            r.raise_for_status()
            parser = MyHTMLParser(url)
            parser.feed(r.text)

            for resulturl, result in parser.fileList:
                # print('Results: {} : {}'.format(resulturl, result))
                tbef, taft, mission = fileToRange(os.path.basename(result))

                if (tbef <= slc_start_dt) and (taft >= slc_end_dt):
                    matchFileName = result
                    parse_url = urllib.parse.urlsplit(resulturl)
                    # Add credentials for osaka
                    new_netloc = auth_netloc_fmt.format(credentials[0],credentials[1],parse_url.netloc)
                    match = urllib.parse.urlunsplit(parse_url._replace(netloc=new_netloc))
                else:
                    print("no match.")
            
            if match is not None:
                found = True
        except Exception as e:
            print("Exception {}".format(e))
        
        if found:
            break

    # print("returning {} : {}".format(matchFileName, match))
    return matchFileName, match


def get_orbit_files():
    import json
    import os
    import osaka
    import osaka.main
    orbit_dates = []
        
    for slc in ctx["localize_slcs"]:
        match = SLC_RE.search(slc)
        if not match:
            raise RuntimeError("Failed to recognize SLC ID %s." %slc)
        mission = match.group('mission')
        slc_start_dt_str = "{}-{}-{}T{}:{}:{}".format(match.group('start_year'), 
                                                      match.group('start_month'),
                                                      match.group('start_day'),
                                                      match.group('start_hour'),
                                                      match.group('start_min'),
                                                      match.group('start_sec'))
        slc_start_dt = datetime.strptime(slc_start_dt_str, "%Y-%m-%dT%H:%M:%S")
        slc_end_dt_str = "{}-{}-{}T{}:{}:{}".format(match.group('end_year'), 
                                                      match.group('end_month'),
                                                      match.group('end_day'),
                                                      match.group('end_hour'),
                                                      match.group('end_min'),
                                                      match.group('end_sec'))
        slc_end_dt = datetime.strptime(slc_end_dt_str, "%Y-%m-%dT%H:%M:%S")
        day_dt_str = slc_start_dt.strftime('%Y-%m-%d')
        if day_dt_str not in orbit_dates:
            orbit_dates.append(day_dt_str)
            orbit_file_name, orbit_file_uri = get_download_orbit_dict(slc_start_dt, slc_end_dt, mission)
            if orbit_file_uri is not None:
                directory = os.path.join(wd, "orbits")
                if not os.path.exists(directory):
                    os.makedirs(directory)
                    print(f"created {directory}")
                osaka.main.get(orbit_file_uri, os.path.join(directory, orbit_file_name))
                print("Downloaded orbit file {}".format(orbit_file_name))
            else:
                print('No orbit files found for slc: {}'.format(slc))   

In [ ]:
def get_current_time():
    import datetime
    return datetime.datetime.now()

def download_slc(slc_id, path):
    
    
    url = "https://datapool.asf.alaska.edu/SLC/SA/{}.zip".format(slc_id)
    print("Downloading {} : {}".format(slc_id, url))
    
    if not os.path.exists(path):
        os.makedirs(path)
    osaka.main.get(url, path)

def run_cmd_output(cmd):
    from subprocess import check_output, CalledProcessError
    cmd_line = " ".join(cmd)
    print("Calling: {}".format(cmd_line))
    output = check_output(cmd_line, shell=True)
    return output

def run_cmd(cmd):

    import subprocess
    from subprocess import check_call, CalledProcessError
    import sys
    cmd_line = " ".join(cmd)
    print("Calling : {}".format(cmd_line))
    p = subprocess.Popen(cmd_line, shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    while True: 
        line = p.stdout.readline()
        if not line:
            break
        print(line.strip())
        sys.stdout.flush()
        
def get_minimum_bounding_rectangle(ctx):
    from math import floor, ceil

    slc_ids = [x for x in os.listdir(os.path.join(ctx["work_dir"],'zip')) if '.zip' in x and '_SLC__' in x]
    #print(slc_ids)
    
    all_north = []
    all_south = []
    all_west = []
    all_east = []
    
    for slc in slc_ids:
        
        zipped = zipfile.ZipFile(os.path.join(wd,'zip',slc), "r")
        overlay = os.path.join(slc.split('.')[0] + '.SAFE', 'preview', 'map-overlay.kml')
        overlay_contents = str(zipped.read(overlay)).split('\\n')


        date_lats = []
        date_lons = []
        for line in overlay_contents:

            if 'coordinates' in line:
                coords_str = line.strip().split('>')[1].split('<')[0]
                coords_list = coords_str.split()
            
                for pair in coords_list:

                    date_lats.append(float(pair.split(',')[1]))
                    date_lons.append(float(pair.split(',')[0]))
        #print()
        #print(date_lats)
        #print(date_lons)
        all_south.append(min(date_lats))
        all_north.append(max(date_lats))
        all_west.append(min(date_lons))
        all_east.append(max(date_lons))
    #print()
    #print(all_lats)
    #print(all_lons)
    min_lat = max(all_south)
    max_lat = min(all_north) 
    min_lon = max(all_west)
    max_lon = min(all_east)

    min_lat_lo = floor(min_lat)
    max_lat_hi = ceil(max_lat)
    min_lon_lo = floor(min_lon)
    max_lon_hi = ceil(max_lon)

    return min_lat, max_lat, min_lon, max_lon, min_lat_lo, max_lat_hi, min_lon_lo, max_lon_hi

def get_user_input_bbox(ctx):
    """
    :param ctx_file: dictionary from cxt file
    :return: void
    """
    from math import floor, ceil
    min_lat = ctx_file['min_lat']
    max_lat = ctx_file['max_lat']
    min_lon = ctx_file['min_lon']
    max_lon = ctx_file['max_lon']

    min_lat_lo = floor(min_lat)
    max_lat_hi = ceil(max_lat)
    min_lon_lo = floor(min_lon)
    max_lon_hi = ceil(max_lon)

    return min_lat, max_lat, min_lon, max_lon, min_lat_lo, max_lat_hi, min_lon_lo, max_lon_hi

def get_master_date(ctx):
    master_date = ctx.get('master_date', "")
    return master_date

def get_bbox(ctx):
    # min_lat, max_lat, min_lon, max_lon = ctx['region_of_interest']

    if ctx['min_lat'] != "" and ctx['max_lat'] != "" and ctx['min_lon'] != "" and ctx['max_lon'] != "":
        # if all values are present in _context.json we can assume user put them in manually
        bbox_data = get_user_input_bbox(ctx)
    else:
        # if user did not define ANY lat lons
        bbox_data = get_minimum_bounding_rectangle(ctx)
        #print(bbox_data)

    return bbox_data

def download_dem():
    dem_cmd = [
        "{}/applications/dem.py".format(ISCE_HOME), "-a",
        "stitch", "-b", "{} {} {} {}".format(MINLAT_LO, MAXLAT_HI, MINLON_LO, MAXLON_HI),
        "-r", "-s", "1", "-f", "-c", "|", "tee", "dem.txt"
        #"-n", dem_user, "-w", dem_pass,"-u", dem_url
    ]
    run_cmd(dem_cmd)
    
def download_slcs(zipdir):
    
    for slc in ctx["localize_slcs"]:
        slcfile = os.path.join(wd,zipdir,slc+'.zip')
        if not os.path.exists(slcfile):
            print('Downloading ', slcfile)
            download_slc(slc, os.path.join(wd,zipdir))
        else:
            print('Found ', slcfile)


## Download Sentinel-1 data SLCs ####


In [ ]:
zipdir = "zip"
download_slcs(zipdir)

### Find a common area of interest from all listed images

In [ ]:
MINLAT, MAXLAT, MINLON, MAXLON, MINLAT_LO, MAXLAT_HI, MINLON_LO, MAXLON_HI =get_bbox(ctx)
print("{} {} {} {} {} {} {} {}".format(MINLAT, MAXLAT, MINLON, MAXLON, MINLAT_LO, MAXLAT_HI, MINLON_LO, MAXLON_HI))

### Visualize the area of interest

In [ ]:
bbox_poly = [(MINLAT, MINLON),(MINLAT, MAXLON),(MAXLAT,MAXLON),(MAXLAT,MINLON),(MINLAT,MINLON)]

import folium
m = folium.Map(location=[37.718890744427554, -118.88748662532234], zoom_start=8, tiles="Stamen Terrain")
folium.Marker(
    [37.718890744427554, -118.88748662532234], popup="<b>Long Valley Caldera</b>").add_to(m)
folium.Polygon(bbox_poly).add_to(m)

m

## Download Orbit Files based on SLCs ####

In [ ]:
get_orbit_files()

## Download and stitch DEM ####

Download of DEM (need to use wgs84 version) using the ISCE DEM download script.

```
mkdir DEM; cd DEM
dem.py -a stitch -b 18 20 -100 -97 -r -s 1 –c
rm demLat*.dem demLat*.dem.xml demLat*.dem.vrt
cd ..
```


In [ ]:

download_dem()

cwd = os.getcwd()
print("cwd : {}".format(cwd))
if os.path.exists("dem.txt"):
    cmd = ["awk", "'/wgs84/ {print $NF;exit}'", "dem.txt"]
    WGS84 = run_cmd_output(cmd).decode("utf-8").strip()
    wgs84_file = os.path.join(cwd, WGS84)
    print("WGS84 : a{}b".format(wgs84_file))
    if os.path.exists(wgs84_file):
        print("Found wgs84 file: {}".format(wgs84_file))
        fix_cmd = ["{}/applications/fixImageXml.py".format(ISCE_HOME), "--full", "-i", "{}".format(wgs84_file) ]
        run_cmd(fix_cmd) 
    else:
        print("NO WGS84 FILE FOUND : {}".format(wgs84_file))
        

#### AUX_CAL file download ####

The following calibration auxliary (AUX_CAL) file is used for **antenna pattern correction** to compensate the range phase offset of SAFE products with **IPF verison 002.36** (mainly for images acquired before March 2015). If all your SAFE products are from another IPF version, then no AUX files are needed. Check [ESA document](https://earth.esa.int/documents/247904/1653440/Sentinel-1-IPF_EAP_Phase_correction) for details. 

Make sure your aws credentials are fresh (i.e. run 'aws-login -p default' in a terminal window) and run the command below to download the AUX_CAL file once and store it somewhere (_i.e._ ~/aux/aux_cal) so that you can use it all the time, for `stackSentinel.py -a` or `auxiliary data directory` in `topsApp.py`.

```aws s3 cp --recursive s3://nisar-dev-ondemand/S1_aux/  ~/aux/aux_cal/```

In the cell below, the necessary AUX_CAL file is copied to the AuxDir directory from this download.

In [ ]:
%%bash
mkdir -p ./AuxDir
cp ~/aux/aux_cal/S1A/S1A_AUX_CAL_V20140915T100000_G20151125T103928.SAFE/data/s1a-aux-cal.xml ./AuxDir

## STACK SENTINEL DATA


The scripts provides support for Sentinel-1 TOPS stack processing. Currently supported workflows include a coregistered stack of SLC, interferograms, offsets, and coherence. 

`stackSentinel.py` generates all configuration and run files required to be executed on a stack of Sentinel-1 TOPS data. When stackSentinel.py is executed for a given workflow (-W option) a **configs** and **run_files** folder is generated. No processing is performed at this stage. Within the run_files folder different run\_#\_description files are contained which are to be executed as shell scripts in the run number order. Each of these run scripts call specific configure files contained in the “configs” folder which call ISCE in a modular fashion. The configure and run files will change depending on the selected workflow. To make run_# files executable, change the file permission accordingly (e.g., `chmod +x run_01_unpack_slc`).

```bash
stackSentinel.py -H     #To see workflow examples,
stackSentinel.py -h     #To get an overview of all the configurable parameters
```

Required parameters of stackSentinel.py include:

```cfg
-s SLC_DIRNAME          #A folder with downloaded Sentinel-1 SLC’s. 
-o ORBIT_DIRNAME        #A folder containing the Sentinel-1 orbits. Missing orbit files will be downloaded automatically
-a AUX_DIRNAME          #A folder containing the Sentinel-1 Auxiliary files
-d DEM_FILENAME         #A DEM (Digital Elevation Model) referenced to wgs84
```

In the following, different workflow examples are provided. Note that stackSentinel.py only generates the run and configure files. To perform the actual processing, the user will need to execute each run file in their numbered order.

In all workflows, coregistration (-C option) can be done using only geometry (set option = geometry) or with geometry plus refined azimuth offsets through NESD (set option = NESD) approach, the latter being the default. For the NESD coregistrstion the user can control the ESD coherence threshold (-e option) and the number of overlap interferograms (-O) to be used in NESD estimation.


In [ ]:
master_date=get_master_date(ctx)
os.chdir(wd)
print("master_date : {}".format(master_date))

cmd = [
    "stackSentinel.py", "-s", "zip/", "-d", "{}".format(wgs84_file), "-a", "AuxDir/", "-m", "{}".format(master_date), "-o", "orbits", 
    "-b", "\"{} {} {} {}\"".format(MINLAT, MAXLAT, MINLON, MAXLON), 
    "-W", "interferogram", "-C", "geometry"
]
run_cmd(cmd)          
              

In [ ]:
cmd = ["topsstack-hamsar/topsStack/stackSlcDn_run2.5.sh", "{} {} {} {}".format(MINLAT, MAXLAT, MINLON, MAXLON)]
run_cmd(cmd)

## Run all steps ##

In [ ]:
!chmod +x run_files/*

# Step 1
!run_files/run_01_unpack_topo_reference > run1.log;
print('Step 1 Finished!')
print()
# Step 2
!cat run_files/run_02_unpack_secondary_slc | parallel -j2 --eta --load 50% > run2.log;
print('Step 2 Finished!')
print()
# Step 2.5
!cat run_files/run_02.5_slc_noise_calibration | parallel -j2 --eta --load 50% > run2_5.log;
print('Step 2.5 Finished!')
print()
# Step 3
!cat run_files/run_03_average_baseline | parallel -j2 --eta --load 50% > run3.log;
print('Step 3 Finished!')
print()
# Step 4
!cat run_files/run_04_fullBurst_geo2rdr  | parallel -j2 --eta --load 50% > run4.log;
print('Step 4 Finished!')
print()
# Step 5
!cat run_files/run_05_fullBurst_resample  | parallel -j2 --eta --load 50% > run5.log;
print('Step 5 Finished!')
print()
# Step 6
!sh run_files/run_06_extract_stack_valid_region > run6.log;
print('Step 6 Finished!')
print()
# Step 7
!cat run_files/run_07_merge_reference_secondary_slc | parallel -j2 --eta --load 50% > run7.log;
print('Step 7 Finished!')
print()
# Step 8
!cat run_files/run_08_generate_burst_igram | parallel -j2 --eta --load 50% > run8.log;
print('Step 8 Finished!')
print()
# Step 9
!cat run_files/run_09_merge_burst_igram | parallel -j2 --eta --load 50% > run9.log;
print('Step 9 Finished!')
print()
# Step 10
!cat run_files/run_10_filter_coherence | parallel -j2 --eta --load 50% > run10.log;
print('Step 10 Finished!')
print()
# Step 11
!cat run_files/run_11_unwrap | parallel -j2 --eta --load 50% > run11.log;
print('Step 11 Finished!')
print()

## Display output products

In [ ]:
ifgdir = os.path.join(wd,'merged','interferograms')
ifglist = os.listdir(ifgdir)

In [ ]:
# Go through interferogram folder and plot things.
ifgdir = os.path.join(wd,'merged','interferograms')
ifglist = os.listdir(ifgdir)

for i in ifglist:
    
    # Wrapped Interferograms
    wifg = os.path.join(ifgdir,i,'filt_fine.int')
    ds = gdal.Open(wifg)
    arr = ds.GetRasterBand(1).ReadAsArray()
    ph = np.angle(arr)
    arrx = arr.shape[1]
    arry = arr.shape[0]
    xx = np.arange(arrx)
    yy = np.arange(arry)
    
    plt.figure(figsize=(10,10))
    plt.pcolormesh(xx, yy, ph,shading='nearest',cmap='jet');
    plt.title(i + ' - wrapped')
    
    outfig = os.path.join(ifgdir,i, i + '_wrap_ra.png')
    plt.savefig(outfig)
    plt.close()


    # Unwrapped Interferograms
    uifg = os.path.join(ifgdir,i,'filt_fine.unw')
    ds = gdal.Open(uifg)
    arr = ds.GetRasterBand(2).ReadAsArray()
    arrx = arr.shape[1]
    arry = arr.shape[0]
    xx = np.arange(arrx)
    yy = np.arange(arry)
    
    plt.figure(figsize=(10,10))
    plt.pcolormesh(xx, yy, arr,shading='nearest',cmap='jet');
    plt.title(i + ' - unwrapped')
    
    outfig = os.path.join(ifgdir,i,i + '_unw_ra.png')
    plt.savefig(outfig)
    plt.close()

    # Correlation
    wcorr = os.path.join(ifgdir,i,'filt_fine.cor')
    ds = gdal.Open(wcorr)
    arr = ds.GetRasterBand(1).ReadAsArray()
    arrx = arr.shape[1]
    arry = arr.shape[0]
    xx = np.arange(arrx)
    yy = np.arange(arry)
    
    plt.figure(figsize=(10,10))
    plt.pcolormesh(xx, yy, arr,shading='nearest',cmap='gray');
    plt.title(i + ' - correlation')
    
    outfig = os.path.join(ifgdir,i,i + '_cor_ra.png')
    plt.savefig(outfig)
    plt.close()


In [ ]:
ifgdir = os.path.join(wd,'merged','interferograms')
ifglist = os.listdir(ifgdir)
ifglist

### Display wrapped interferograms

In [ ]:
from IPython.display import Image
from ipywidgets import interact

ifg_ra = []
for i in ifglist:
    for f in os.listdir(os.path.join(ifgdir,i)):
        #print(f)
        if 'png' in f and 'wrap' in f:
            ifg_ra.append(os.path.join(ifgdir,i,f))
@interact
def show_images(file=ifg_ra):
    display(Image(file))
    
    

### Display unwrapped interferograms

In [ ]:
ifg_ra = []
for i in ifglist:
    for f in os.listdir(os.path.join(ifgdir,i)):
        #print(f)
        if 'png' in f and 'unw' in f:
            ifg_ra.append(os.path.join(ifgdir,i,f))
@interact
def show_images(file=ifg_ra):
    display(Image(file))
    
    

### Display correlation plots

In [ ]:
ifg_ra = []
for i in ifglist:
    for f in os.listdir(os.path.join(ifgdir,i)):
        #print(f)
        if 'png' in f and 'cor' in f:
            ifg_ra.append(os.path.join(ifgdir,i,f))
@interact
def show_images(file=ifg_ra):
    display(Image(file))
    
    
    

In [ ]:
# Interferograms
for i in ifglist:
    for f in os.listdir(os.path.join(wd,'merged','interferograms',i)):
        if f.split('.')[-1]=='unw':
            os.chdir(os.path.join(wd,'merged','interferograms',i))
            comm = 'geocodeGdal.py  -l ' + os.path.join(wd,'merged','geom_reference','lat.rdr') \
            + ' -L ' + os.path.join(wd,'merged','geom_reference','lon.rdr') \
            + ' -f '+ os.path.join(wd,'merged','interferograms',i,f) \
            + ' -b "' + str(MINLAT) + ' ' + str(MAXLAT) + ' ' + str(MINLON) + ' ' + str(MAXLON) + '"'
            print(comm)
            os.system(comm)

os.chdir(wd)       